In [ ]:
import os
import sys
sys.path.append("..")
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

pd.options.display.precision = 15
data_dir = "D:/KaggleData/earthquakes/"
plt.style.use("dark_background") # uncomment if using light jupyter theme

Load the data

In [ ]:
train = pd.read_csv(os.path.join(data_dir, "train.csv"),
                    dtype={"acoustic_data": np.int16, "time_to_failure": np.float64})

train.head()

In [ ]:
print("We have {:,} rows and {} columns.".format(train.shape[0], train.shape[1]))

Let's save the column names as variables for convenience:

In [ ]:
xcol = "acoustic_data"
ycol = "time_to_failure"

Test data has a different structure: there is a separate CSV for each segment. Let's look at a couple of them:

In [ ]:
# take the segment ids from the sample submission file
sample_submission = pd.read_csv(os.path.join(data_dir, "sample_submission.csv"), index_col="seg_id")
x_test = pd.DataFrame(columns=[xcol], dtype=np.float64, index=sample_submission.index)

# load first 10 segments
for seg_id in x_test.index[0:10]:
    segment = pd.read_csv(os.path.join(data_dir, "test", seg_id + ".csv"))
    print("shape: {}".format(segment.shape))

Test data consists of segments of 150,000 observations each.

# 1. EDA

## 1.1. High level stats

In [ ]:
train.describe()

In [ ]:
segment.describe()

Data is centered around 4 or 5, but has big outlier values (high as well as low). Let's look at this closer.

In [ ]:
print("Number of earthquakes is {}".format(np.sum(train[ycol] == 0)))

## 1.2 Distributions

Let's see where our signal values are.

In [ ]:
train[xcol].hist(bins=np.arange(-20, 30, 1))

They show a nice Gaussian distribution in general, but note that we cut off the outliers in this plot. Plot those separately:

In [ ]:
def plot_quantiles(data, q, ax):
    y = np.quantile(data, q=q)
    return ax.plot(q, y)

fig, axes = plt.subplots(1, 2, figsize=(10, 5))
x1 = np.arange(0.95, 1.0, 0.005)
x2 = np.arange(0.95, 0.995, 0.005)
axes[0] = plot_quantiles(train[xcol].values, x1, axes[0])
axes[1] = plot_quantiles(train[xcol].values, x2, axes[1])
plt.show()


The highest values are way, way higher than all others: as seen in some kernels, these happen right before an earthquake.

In [ ]:
train[ycol].hist()

# 2. Creating training data similar to test data

Our training data is one big, continuous sequence of observations, while ttthe test data consists of smaller segments of 150,000 observations each. In order to train in the same way that we need to predict, we should make the datasets more similar. Firstly, we need to create sequences of train data similar to the test sequences. This can be done in multiple ways:
1. The easiest way to do this is simply to cut the training data in pieces of 150,000 sequential observations, i.e., take values 0 to 149999, 150000 to 299999, etc.
2. In order to create even more training data, we can make the splits random, i.e., start at random positions in the data and return the next 150,000 data points. This way, the 'cuts' are not made at fixed positions, which might help against overfitting as well.

Let's try the second approach:

In [ ]:
def generate_train_sequence(data, size=150000):
    while True:
        idx = np.random.randint(0, len(data))
        yield data[idx:idx+size]

train_gen = generate_train_sequence(train)

Now you can generate as many training sequences as you want. For illustration:

In [ ]:
n_samples = 5  # make this bigger of course
for i in range(n_samples):
    sample = next(train_gen)
    # ...do training on sample or store it...
    print(sample.shape)

Secondly, for every sequence of 150,000 obervations, we need to only return one prediction, which should match the time remaining to the earthquake at the __end of the sequence__.
Let's make sure this is what we return, instead of just a slice of the DataFrame. Note: Here we also improve the speed by only calling `np.random.randint()` once every 10,000 indices (there is significant overhead in making that call).

In [ ]:
def sequence_generator(data, xcol="acoustic_data", ycol="time_remaining", size=150000):
    """Generator that extracts segments of the signal from the data."""
    while True:
        indices = np.random.randint(0, len(data) - size - 1, 10000)
        for idx in indices:
            y = data[ycol].iloc[idx + size - 1]
            x = data[idx:(idx + size)][xcol].values
            yield x, y

train_gen = sequence_generator(train, xcol=xcol, ycol=ycol, size=150000)
for i in range(5):
    sample_x, sample_y = next(train_gen)
    print("sample length: {}, prediction: {}".format(len(sample_x), sample_y))

Now, we can create random datasets or batches of samples to train models on. However, storing the 150,000 observations for every training sample is gonna lead to problems. Hence, it is better to first compute features on the generated samples, and store only the computed features.

Let's create a simple class that computes features and apply it to randomly sampled sequences of data.

In [ ]:
class FeatureComputer():

    feats = ["minimum", "maximum", "mean", "median", "std"]

    def __init__(self, minimum=True, maximum=True, mean=True, median=True, std=True, quantiles=None, verbose=True):
        self.minimum = minimum
        self.maximum = maximum
        self.mean = mean
        self.median = median
        self.std = std
        self.quantiles = quantiles
        
        self.feature_names = self._infer_names()
        self.n_features = np.sum([minimum, maximum, mean, median, std, len(quantiles)])
        self.result_template = np.zeros(self.n_features)

        self.verbose = verbose

    def _infer_names(self):
        quantile_names = [str(q) + "-quantile" for q in self.quantiles]
        i = 0
        names = np.array(self.feats)[[self.minimum, self.maximum, self.mean, self.median, self.std]]
        names = names.tolist() + quantile_names
        return names
        
    def compute(self, arr):
        result = np.zeros_like(self.result_template)
        i = 0
        if self.minimum:
            result[i] = np.min(arr)
            i += 1
        if self.maximum:
            result[i] = np.max(arr)
            i += 1
        if self.mean:
            result[i] = np.mean(arr)
            i += 1
        if self.median:
            result[i] = np.median(arr)
            i += 1
        if self.std:
            result[i] = np.std(arr)
            i += 1
        if self.quantiles is not None:
            result[i:] = np.quantile(arr, q=self.quantiles)
        return result

Example use:

In [ ]:
computer = FeatureComputer(quantiles=[0.0025, 0.005, 0.01, 0.02, 0.05, 0.95, 0.98, 0.99, 0.995, 0.9975])
feat_names = computer.feature_names

for i in range(2):
    sample_x, sample_y = next(train_gen)
    sample_features = computer.compute(sample_x)
    print([feat_names[i] + ": {}".format(sample_features[i]) for i in range(len(sample_features))])


This way, we can create a new dataset with several features and as much samples as we want.

In [ ]:
def create_feature_dataset(data, feature_computer, xcol="acoustic_data", ycol="time_to_failure", n_samples=100):
    
    new_data = pd.DataFrame({feature: np.zeros(n_samples) for feature in feature_computer.feature_names})
    targets = np.zeros(n_samples)
    data_gen = sequence_generator(train, xcol=xcol, ycol=ycol, size=150000)

    for i in range(n_samples):
        x, y = next(data_gen)
        new_data.iloc[i, :] = feature_computer.compute(x)
        targets[i] = y

    new_data[ycol] = targets
    return new_data

generated_train = create_feature_dataset(train, computer, n_samples=10)
generated_train

Let's see if this works okay timewise:

In [ ]:
from timeit import timeit

def time_sampling():
    create_feature_dataset(train, computer, n_samples=1000)

avg_time = timeit(time_sampling, number=5) / 5
print("It takes about {:.2f} seconds to sample and process 1000 sequences. "
      "Note that this is 150 million observations ({:.2f}% of the dataset)."
      .format(avg_time, 150e6/train.shape[0]*100))

# 3. Adding Cross-Validation
However, we cannot sample validation sequences from the same dataset as the training samples, since there might be overlap (data leakage) in the sampled sequences. Hence, we need to split first. Note that the data should remain in the same order when we split; we should not shuffle, because then we destroy the time series.

Let's create a simple cross validation framework.

In [ ]:
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
import time # for loggin progress


# helper function to print progress with time stamp
def progress(text, verbose=True, same_line=False, newline_end=True):
    if verbose:
        print("{}[{}] {}".format("\r" if same_line else "", time.strftime("%Y-%m-%d %H:%M:%S"),
                                 text), end="\n" if newline_end else "")


def train_and_predict(train_x, train_y, val_x, model_cls, **model_params):
    model = model_cls(**model_params)
    model.fit(train_x, train_y)
    predictions = model.predict(val_x)
    return predictions


def cross_validate(data, model_cls, feature_computer, ycol="time_to_failure", n_splits=5, 
                   train_samples=1000, val_samples=500, **model_params):
    """Perform custom cross validation using randomly sequences of observations."""
    splitter = KFold(n_splits=n_splits, shuffle=False)

    scores = []
    for i, (train_index, val_index) in enumerate(splitter.split(data)):
        progress("Starting cross-validation fold {}.".format(i))

        # split the data according to the indices
        progress("Splitting data in train and validation sets.")
        train = data.iloc[train_index]
        val = data.iloc[val_index]

        # sample random sequences for training
        progress("Sampling {} sequences from training data.".format(train_samples))
        train_features = create_feature_dataset(train, feature_computer, n_samples=train_samples)
        y_train = train_features[ycol]
        x_train = train_features.drop(ycol, axis=1)
        progress("Train set sampled.")

        # sample random sequences for validation
        progress("Sampling {} sequences from validation data.".format(val_samples))
        val_features = create_feature_dataset(val, feature_computer, n_samples=val_samples)
        y_val = val_features[ycol]
        x_val = val_features.drop(ycol, axis=1)
        progress("Validation set sampled.")

        # train and predict validation set
        progress("Start training and predicting.")
        y_val_hat = train_and_predict(x_train, y_train, x_val, model_cls, **model_params)
        progress("Predictions on validation set made.")
        
        # evaluate using mean absolute error for this competition
        score = mean_absolute_error(y_val, y_val_hat)
        scores.append(score)
        progress("Validation score: {}.".format(score))

    return scores

Perform Cross Validation:

In [ ]:
params = {
    "n_estimators": 1000,
    "criterion": 'mae',
    "n_jobs": -1,
    "verbose": 1,
}

scores = cross_validate(train, RandomForestRegressor, computer, **params)

In [ ]:
print("Mean validation score: {}".format(np.mean(scores)))

Let's train the model on a bunch of samples from the entire training data and predict the test to see whether the leaderboard and cross validation score are similar. So no blending or stacking for now..

In [ ]:
def predict_on_test(model, feature_computer, ycol="time_to_failure", data_dir=data_dir):

    # take the segment ids from the sample submission file
    sample_submission = pd.read_csv(os.path.join(data_dir, "sample_submission.csv"), index_col="seg_id")
    x_test = pd.DataFrame(columns=feature_computer.feature_names, dtype=np.float64, index=sample_submission.index)

    # load and predict segments one by one
    for i, seg_id in enumerate(x_test.index):
        progress("Loading and computing features for segment {}/{}.".format(i + 1, len(x_test)),
                 same_line=True, newline_end=(i + 1 == len(x_test)))

        segment = pd.read_csv(os.path.join(data_dir, "test", seg_id + ".csv"))
        x_test.loc[seg_id, :] = feature_computer.compute(np.array(segment))

    sample_submission[ycol] = model.predict(x_test)
    progress("Predictions made.")
    return sample_submission

Train a Random Forest

In [ ]:
model = RandomForestRegressor(**params)

progress("Creating dataset of 5000 training samples.")
train_features = create_feature_dataset(train, computer, n_samples=5000)

progress("Fitting RandomForestRegressor on data.")
model.fit(train_features.drop(ycol, axis=1), train_features[ycol])

And predict on the train data.

In [ ]:
progress("Loading and predicting test data.")
submission = predict_on_test(model, computer)
submission.reset_index().head()

In [ ]:
submission.reset_index().to_csv(os.path.join(data_dir, "first_submission.csv"), index=False)

__Leaderboard score: 1.758 (place 868 / 1200 at time of submitting).__
Score is better than CV score of 2.2. This may be due to the fact that we have actual earthquakes in the training (and thus validation) data and not in the test data.

Ways of improvement:
- Remove observations around earthquakes from the training data to better resemble the test data.
- Calculate more features:
    - calculate features over subsequences of a 150k sequence, e.g., calc features for every 15k observations, so that we have timely information.
    - calculate more typical signal processing features, e.g., Fourier Transforms and stuff..
    - other features...
    - Use RNNs to learn features from the raw signal.
- We are underfitting: use more complex models.
- Tune the model(s).
- Use blending (or stacking but blending is easier and works better for this competition according to some kernel).
- Fix random seeds to get more stable output (since now we have a different dataset at every run).